In [ ]:
#Prepare Data From CSV Recording

##Load File

'Data is loaded from a CSV recording file, accepted through an input prompt. This includes all positional data related to the 6 trackers (HMD, Left Controller, Right Controller, Waist, Left Foot, Right Foot).'

'Data is loaded into a Pandas dataframe. The primary tracking data is then extracted, leaving extraneous data such as booleans for button presses.'

'The extracted columns are then concatenated into a new dataframe, and the columns are renamed for ease of reading.' 

'The new trimmed file is written to a directory (/trim_output), for further manipulation and loading into the model.'


In [1]:
import numpy as np 
import pandas as pd

recording_path = "../recordings/"
output_path = "../trim_output/"
file_name = input("Input File Name")

#Read in CSV

try:
    dataframe = pd.read_csv(recording_path + file_name + ".csv")
except:
    print("Error Reading File: Check Spelling and Try Again")

# Seperate each tracker to seperate dataframe
HMD = dataframe.iloc[:, 1:4]
controller_1 = dataframe.iloc[:, 7:10]
controller_2 = dataframe.iloc[:, 37:40]
left_foot = dataframe.iloc[:, 67:70]
right_foot = dataframe.iloc[:, 73:76]
waist = dataframe.iloc[:, 79:82]

# Join all trackers together
joined = pd.concat([HMD, controller_1, controller_2,
                   waist, left_foot, right_foot], axis=1)

# set new column headers
joined.columns = [
    "head_x",
    "head_y",
    "head_z",
    "r_controller_x",
    "r_controller_y",
    "r_controller_z",
    "l_controller_x",
    "l_controller_y",
    "l_controller_z",
    "waist_x",
    "waist_y",
    "waist_z",
    "r_foot_x",
    "r_foot_y",
    "r_foot_z",
    "l_foot_x",
    "l_foot_y",
    "l_foot_z"
]


# output to new csv
output_file = output_path + file_name + "_trimmed.csv"
joined.to_csv(output_file, index=False)

print(file_name + " output to " + output_path)


Input File Name l


Error Reading File: Check Spelling and Try Again


NameError: name 'dataframe' is not defined

In [ ]:
#Data Normalization

## Data Scaling

'The new CSV is loaded into memory, chosen through an input prompt'
'The data is then split between the features (the HMD and controller tracking data), and the labels (the waist and foot trackers).'
'These are loaded into Numpy arrays to peform normaliztion. The output from OpenVR Recorder is upscaled by 100. To correct this the array is divided by 100'

In [3]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
#from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
#import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


output_path = "../trim_output/"

file_name = input("Input File Name")

#read in formatted CSV
try:
    dataframe = pd.read_csv(output_path + file_name + ".csv")
    print("Dataframe created")
except:
    print("Error Reading File")


#Show shape of the dataframe
dataframe.describe()

x_train_df = dataframe.iloc[:, 0:9]
y_train_df = dataframe.iloc[:, 9:18]
#display(x_train_df)
#display(y_train_df)

#Load data into Numpy array
x_train = np.array(x_train_df)
y_train = np.array(y_train_df)



#Divide values in array by 100
x_train_normalized = np.divide(x_train, 100)
y_train_normalized = np.divide(y_train, 100)

print(x_train_normalized.shape, x_train_normalized)
print(y_train_normalized.shape, y_train_normalized)

#x_train, x_test, y_train, y_test = train_test_split(x_train_normalized, y_train_normalized)








Input File Name walking_1_train


Dataframe created
(972, 9) [[ 0.00396347  1.55321396 -0.09726781 ... -0.17044104  0.79216263
  -0.12728346]
 [ 0.00396347  1.55321396 -0.0974095  ... -0.1698889   0.79222771
  -0.12750142]
 [ 0.00419568  1.55321396 -0.09760312 ... -0.16916578  0.79246605
  -0.12798414]
 ...
 [-0.07326494  1.57561111 -0.15563388 ... -0.28583529  0.76833069
   0.04462518]
 [-0.07258665  1.57578445 -0.15483627 ... -0.28606691  0.76840103
   0.04523792]
 [-0.07203475  1.57610062 -0.15404955 ... -0.28627296  0.76849533
   0.04572279]]
(972, 9) [[ 0.03550047  0.99325623 -0.06203985 ... -0.17026985  0.09899879
   0.01698667]
 [ 0.03546809  0.99353798 -0.06230913 ... -0.17021811  0.09892341
   0.01698667]
 [ 0.0358079   0.99352867 -0.06240509 ... -0.17014109  0.09882551
   0.01698667]
 ...
 [-0.13245131  1.00056854 -0.13201273 ... -0.08991505  0.09783401
   0.05129308]
 [-0.13226092  1.00068802 -0.13195515 ... -0.09031304  0.09860064
   0.05091204]
 [-0.13171673  1.00074806 -0.13161671 ... -0.08997025  0.09833

In [8]:

x_train_normalized.shape

x_train_reshaped = np.expand_dims(x_train_normalized, axis=0)
print(x_train_reshaped.shape)
display(x_train_reshaped)

(1, 972, 9)


array([[[ 0.00396347,  1.55321396, -0.09726781, ..., -0.17044104,
          0.79216263, -0.12728346],
        [ 0.00396347,  1.55321396, -0.0974095 , ..., -0.1698889 ,
          0.79222771, -0.12750142],
        [ 0.00419568,  1.55321396, -0.09760312, ..., -0.16916578,
          0.79246605, -0.12798414],
        ...,
        [-0.07326494,  1.57561111, -0.15563388, ..., -0.28583529,
          0.76833069,  0.04462518],
        [-0.07258665,  1.57578445, -0.15483627, ..., -0.28606691,
          0.76840103,  0.04523792],
        [-0.07203475,  1.57610062, -0.15404955, ..., -0.28627296,
          0.76849533,  0.04572279]]])

#Model Creation and Training

In [17]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(x_train_normalized), output_dim= 16, input_length=9 ),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 9, 16)             14480     
                                                                 
 bidirectional_2 (Bidirectio  (None, 64)               9600      
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 10)                650       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 24,741
Trainable params: 24,741
Non-trainable params: 0
_________________________________________________________________


In [19]:

# fit the model
model.fit(x_train_normalized, y_train_normalized, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(x_train_normalized, y_train_normalized, verbose=0)
print('Accuracy: %f' % (accuracy*100))

ValueError: in user code:

    File "C:\Users\Jev\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Jev\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Jev\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Jev\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\Jev\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "C:\Users\Jev\AppData\Local\